### Object tracking with opencv

In [1]:
import cv2

In [2]:
def initialize(videopath):
    cap = cv2.VideoCapture(videopath)
    od = cv2.createBackgroundSubtractorMOG2(varThreshold=50, history=100) # better selection and quicker
    # od = cv2.createBackgroundSubtractorKNN()
    return cap, od

In [3]:
cap, od =initialize('../data/aerial_football_video.mp4')

In [4]:
tracker = cv2.legacy.TrackerMOSSE_create()
init_bb = None
start = None
count = 0

In [5]:
while True:
    ret, frame = cap.read()
    count += 1
    if not ret:
        print('ending run')
        break
    key = cv2.waitKey(1)
    if key == ord('q'):                                         # exit
        break        
    elif key == ord('x'):                                       # pauze screen
        cv2.waitKey(0)            
    frame = cv2.resize(frame, (960, 540))                       # Resize image
    cv2.imshow('frame', frame)    
    mask = od.apply(frame)                                      # run mask (black and white)
    _, mask = cv2.threshold(mask, 254, 255,cv2.THRESH_BINARY)   # set treshold to only get really contracting values in themask
    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) #Each individual contour is a Numpy array of (x,y) coordinates of boundary points of the objects.
    # for cnt in contours:
    #     x,y,w,h = cv2.boundingRect(cnt)
    #     cv2.rectangle(mask, (x, y), (x + w, y + h), (0, 255, 0), 2)  
    # cv2.imshow('mask', mask)
    if init_bb is None and len(contours) > 0:
        for cnt in contours:
            if start is None and count > 10:
                x,y,w,h = cv2.boundingRect(cnt)
                init_bb = (x, y, x+ w, y+h) # check this
                tracker.init(frame, init_bb)   
                start = 'done'
    if init_bb is not None:
        success, box = tracker.update(frame)
        if success:
            (x, y, w, h) = [int(v) for v in box]
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)         
    cv2.imshow('Real-time Object Detection and Tracking', frame)

cap.release()
cv2.destroyAllWindows()
[cv2.waitKey(1) for i in range(1,5)]           


[-1, -1, -1, -1]

In [ ]:
# Initialize variables
initBB = None
while True:
    # Read the frame
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Draw rectangles around detected faces
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Initialize the tracker
    if initBB is None and len(faces) > 0:
        initBB = (faces[0][0], faces[0][1], faces[0][2], faces[0][3])
        tracker.init(frame, initBB)

    # Update the tracker and draw the tracking box
    if initBB is not None:
        success, box = tracker.update(frame)
        if success:
            (x, y, w, h) = [int(v) for v in box]
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)

    # Display the resulting frame
    cv2.imshow('Real-time Object Detection and Tracking', frame)

    # Exit the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close all windows
cap.release()
cv2.destroyAllWindows()

In [6]:
## first step was finding objects, now its time for tracking

In [7]:
def trackng_objects(c, od):
    frame_count = 0
    while True:
        ret, frame = c.read()                                       # read frames
        frame_count += 1
        frame = cv2.resize(frame, (960, 540))                       # Resize image
        if not ret:
            print("Stream ended. Exiting...")
            break        
        key = cv2.waitKey(1)
        if key == ord('q'):                                         # exit
            break        
        elif key == ord('x'):                                       # pauze screen
            cv2.waitKey(0)        
        cv2.imshow('frame', frame)
        mask = od.apply(frame)                                      # run mask (black and white)
        _, mask = cv2.threshold(mask, 254, 255,cv2.THRESH_BINARY) # set treshold to only get really contracting values in themask
        cv2.imshow('mask', mask)
        bbox = cv2.selectROI(frame, False)                        # handmatige select bbox op video
        if frame_count == 1:
            tracker.init(frame, bbox)
            print('initializing tracker')
            print(bbox, tracker)        
        contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) #Each individual contour is a Numpy array of (x,y) coordinates of boundary points of the object.
        detect = []
        for cnt in contours:
            if frame_count > 1:
                x,y,w,h = cv2.boundingRect(cnt)                         # get boundingbox
                ret, bbox = tracker.update(frame)                      
                print(bbox)
                cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2, 1)

                # cv2.rectangle(frame,(x,y), (x+w, y+h),(0,0, 255), 3)     # draw rectangle
                cv2.imshow('frame2', frame)
                frame_count += 1
        # box_ids = tracker.update(detect)
        # for box_id in box_ids:
        #     x,y,w,h, id = box_id
        #     cv2.putText(frame, str(id), (x, y-15), cv2.FONT_HERSHEY_PLAIN, 2, (0,0,255),2)
        #     cv2.rectangle(frame, (x,y), (x+w, y+h), (0, 0, 255), 3)
        #     print(box_ids)
    c.release()
    cv2.destroyAllWindows()
    [cv2.waitKey(1) for i in range(1,5)]                            # needed to destroy window
    return c, frame

In [8]:
trackng_objects(cap, od)

Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!
initializing tracker
(383, 362, 26, 32) < cv2.legacy.TrackerMOSSE 0x7fc0a96ddb70>
Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!
Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!
Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!


In [ ]:
### to do:

## define in first frame the bbox of specific obecjt
## create tracker for that bbox
## update tracker per frame